### Домашка 4

1) Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news
Для парсинга использовать xpath. Структура данных должна содержать:
название источника,
наименование новости,
ссылку на новость,
дата публикации

2) Сложить все новости в БД

In [1]:
from lxml import html
import requests
import datetime

In [2]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

In [3]:
from pymongo import MongoClient
from pymongo import errors

client = MongoClient('localhost', 27017)
db = client['news']

In [4]:
def add_new_into_db(collection, new: dict):
    """
    Функция добавляет одну новость в базу данных монго
    """
    
    if collection.count_documents({'href': new['href']}) == 0:
        collection.insert_one(new)
    

#### lenta.ru

In [40]:
response = requests.get('https://lenta.ru/', headers=header)
dom = html.fromstring(response.text)

items = dom.xpath("//div[contains(@class, 'js-main__content')]/section/div[2]/div")
for item in items:
    
    new = {}
    
    tittle = item.xpath(".//a[contains(@href, '/news/2')]/text()") 
    href = item.xpath(".//a[contains(@href, '/news/2')]/@href") 
    
    if len(tittle) == 0:
        continue
        
    new['source'] = 'lenta.ru'
    new['tittle'] = tittle[0].replace('\xa0', ' ')
    new['href'] = 'https://lenta.ru' + href[0]
    new['date'] = datetime.datetime.strptime(href[0][6:16], '%Y/%m/%d')     

    add_new_into_db(db.lenta_ru, new)
    

#### news.mail.ru

In [39]:
response = requests.get('https://news.mail.ru/', headers=header)
dom = html.fromstring(response.text)

news = []
items = dom.xpath("//ul[contains(@data-module, 'TrackBlocks')]/li")
for item in items:
    
    new = {}
    
    tittle = item.xpath(".//a/text()") 
    href = item.xpath(".//a/@href")

    
    new['tittle'] = tittle[0].replace('\xa0', ' ')
    new['href'] = 'https://news.mail.ru/' + href[0]
    
    response_item = requests.get(new['href'], headers=header)
    dom_item = html.fromstring(response_item.text)
    
    date = dom_item.xpath("//div[contains(@class, 'breadcrumbs_article')]//span[contains(@datetime, '20')]/@datetime") 
    source = dom_item.xpath("//div[contains(@class, 'breadcrumbs_article')]//span[contains(@class, 'link__text')]/text()")
    
    if len(date) == 0:
        new['date'] = datetime.datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
    else:
        new['date'] = datetime.datetime.strptime(date[0][:10], '%Y-%m-%d')  
    
    new['source'] = source[0]
    
    add_new_into_db(db.mail_news, new)
    
    

IndexError: list index out of range

#### yandex.ru/news

In [33]:
response = requests.get('https://yandex.ru/news/', headers=header,)
dom = html.fromstring(response.text)

new = []
items = dom.xpath("//div[contains(@class, 'story__topic')]")
for item in items:
    
    new = {}
    tittle = item.xpath(".//h2/a/text()")
    href = item.xpath(".//h2/a/@href")
    
    story_date = item.xpath("./..//div[contains(@class, 'story__date')]/text()")

    new['tittle'] = tittle[0]
    new['href'] = 'https://yandex.ru' + href[0][0:href[0].find('?')]
    new['source'] = source = story_date[0].replace(' вчера', '')[:-6]
    
    today = datetime.datetime.today()
    today = today.replace(hour=0, minute=0, second=0, microsecond=0)
    if  'вчера' not in story_date:
        new['date'] = today - datetime.timedelta(1)
    else:
        new['date'] = today

    add_new_into_db(db.yandex_news, new)

